In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import copy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

# El Mundo Today

In [4]:
url = 'https://www.elmundotoday.com/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] 
titulo = [] 
por = [] 
cab = [] 
com = []
texto = [] 
pag = [] 
url = [] 


for i in range(0, len(soup.findAll('h3'))):
    web.append('El Mundo Today')
    titulo.append(soup.findAll('h3')[i].text)
    
    # Cabecera: si es la primera noticia que ponga un 1, para el resto, 0
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
    
    # Comentarios = None, no hay comentarios en esta página
    com.append(None)
    
    # Página siempre a 1
    pag.append(1)
    
    # URL
    urll = soup.findAll('h3')[i].find('a').get('href')
    url.append(urll)
    
    # Autor
    peticion_noticia = requests.get(urll, allow_redirects = False)
    soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
    try:
        por.append(soup_noticia.findAll('a', {'class' : 'tdb-author-name'})[0].text)
    except:
        por.append('Revisar')
    
    # Texto
    try:
        tex = []

        for i in range(0, len(soup_noticia.findAll('p', {'class': ''}))):
            tex.append(soup_noticia.findAll('p', {'class': ''})[i].text)

        tex2 = ' '.join(tex)
        texto.append(tex2.split('(function($)')[0])
    except:
        texto.append(None) # Hay muchos vídeos, que no tienen autor declarado en la web

    

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}

emt = pd.DataFrame(table_dict) 

# Maldita 

Web que se dedica a destapar bulos. Como la mayoría de sus artículos son como: "No, el Hospital Arnau de Vilanova de Lleida no está vacío y la carpa que se ha instalado no es un hospital de campaña, funciona como una sala de espera para pacientes con síntomas de COVID-19", es decir, con la estructura "**No,** esto **no** es cierto", la estrategia es sacar todos los titulares eliminado los "no/ni" para quedarnos con una aproximación de los titulares de las noticias falsas.  
Noticia real: "No, esto no es cierto"  
Noticia falsa: "~~No,~~ esto ~~no~~ es cierto" --> "esto es cierto"

In [5]:
url = 'https://maldita.es/malditobulo/'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] 
titulo = [] 
por = [] 
cab = [] 
com = [] 
texto = [] 
pag = [] 
url = [] 

for i in range(0, len(soup.findAll('article'))):
    web.append('Maldita')
    
    try:
        titulo.append(re.sub(r"\bno\b", '', soup.findAll('article')[i]\
                             .find('h2').text.split(':')[0]\
                             .replace('No, ', '').replace(' no ', ' ').replace('No ', '').replace(' ni ',' ')))
    except:
        titulo.append('Revisar')
    
    por.append('Autor')
    cab.append('0')
    pag.append(1)
    url.append(None)
    com.append(None)
    texto.append(None)
    

# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
mb = pd.DataFrame(table_dict)
mb = mb[mb['Título'] != 'Revisar']

## Newtral/bulos

In [6]:
url = 'https://www.newtral.es/tag/bulo/'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] 
titulo = [] 
por = [] 
cab = [] 
com = [] 
texto = [] 
pag = [] 
url = [] 


for i in range(0, len(soup.findAll('article'))):
    web.append('Newtral/bulos')
    por.append('Newtral/bulos')
    cab.append(None)
    com.append(None)
    pag.append(None)
    
    url.append(soup.findAll('article')[i].find('h3').find('a').get('href'))
    
    tex = soup.findAll('article')[i].find('h3').text\
                  .replace('No, ', '').replace(' no ', ' ').replace('No ', '').replace(' ni ',' ').replace('falsas', '')\
                 .replace('falsos', '').replace('falsa', '').replace('falso', '')
    titulo.append(tex)
    texto.append(tex)


# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
nb = pd.DataFrame(table_dict)

## Newtral/fakes

No lo sacamos con BeautifulSoup sino con Selenium (webdriver).

In [7]:
CHROMEDRIVER_PATH = r'C:\Users\ladyl\Downloads\chromedriver_win32\chromedriver.exe'
CHROME_PATH = ""
WINDOW_SIZE = "1920,1080"

chrome_options = Options()
# chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH
prefs = {'profile.managed_default_content_settings.images':2}
chrome_options.add_experimental_option("prefs", prefs)

url = "https://www.newtral.es/fakes/"

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
    chrome_options=chrome_options
    )
driver.get(url)

final = 50
inicial = 1

titular = []

for i in range(inicial, final):

    try:
        for j in range(inicial, inicial + 15):
            titular.append(driver.find_element_by_xpath('//*[@id="main"]/section/div[' + str(j) + ']/article/div[2]/blockquote/div').text)

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)    

        wait = WebDriverWait(driver, 5)
        time.sleep(5)

        inicial = inicial + 15

    except:
        titular.append(None)

web = ['Newtral/Fakes'] * len(titular)
por = [None] * len(titular)
cab = [None] * len(titular)
com = [None] * len(titular)
texto = titular
pag = [None] * len(titular)
url = [None] * len(titular)

table_dict = {'Web': web,\
              'Título': titular,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}

nf = pd.DataFrame(table_dict)

driver.close()

C:\Users\ladyl\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


## El Jueves

In [8]:
url = 'https://www.eljueves.es/news'

# last_page = 70 # en total son 70 páginas
last_page = 2

# Hacer lista con todas las url de cada página de noticias 
list_urls = [url] # la inicializamos con la propia home 

for i in range(2, last_page+1): # range 2 porque la 1 es la home
    list_urls.append(url + '?_page=' + str(i))


# Inicializamos listas vacías para llenarlas con bucles
web = [] 
titulo = [] 
por = [] 
cab = [] 
com = []
texto = [] 
pag = [] 
url = [] 

for n, p in enumerate(list_urls):
    
    peticion = requests.get(p, allow_redirects = False)
    soup = BeautifulSoup(peticion.text, "lxml")

    for i in range(0, len(soup.findAll('article'))):
        web.append('El Jueves')

        # Titular
        try:
            titulo.append(soup.findAll('article')[i].find('h3').text.replace('\r', '').replace('\n', '').replace('\t', ''))
        except:
            titulo.append('Revisar')

        # Autor
        por.append(None)

        # Cabecera
        cab.append(None)

        # Comentarios
        com.append(None)

        # Página
        pag.append(n+1)

        # URL
        urll = 'https://www.eljueves.es' + soup.findAll('article')[i].find('div',  {'class' : "txt"}).find('a').get('href')
        url.append(urll)

        # Texto
        #texto.append('Revisar')
        peticion_noticia = requests.get(urll, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

        tex = []

        try:
            tex.append(soup_noticia.findAll('h2')[0].text.replace('\r', '').replace('\n', '').replace('\t', ''))
        except:
            tex.append('')

        try:
            for j in range(0, len(soup_noticia.findAll('div')[2].findAll('p', {'class' : ''})[4:-1])):
                tex.append(soup_noticia.findAll('div')[2].findAll('p', {'class' : ''})[j+4].text)
            tex = ' '.join(tex)

        except:
            tex.append('Revisar')

        texto.append(tex)

    

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}

ej = pd.DataFrame(table_dict) 

# Dataframe conjunto

In [9]:
# Lista con los dataframes que tenemos de los diferentes periódicos
fake_news = [emt, mb, nb, nf, ej]

# Los unimos, y para que no se repitan los índices, los reseteamos
df = pd.concat(fake_news).reset_index(drop=True)

# Para crear el id (único) de cada noticia
new_id = 'new_' + (df.index + 1).astype(str)

# Y lo insertamos en la primera posición
df.insert(loc=0, column='new_id', value=new_id)

# Ver el dataframe final
df

,new_id,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL
0,new_1,El Mundo Today,"Como cada año, los informativos vuelven a envi...",Kike García,1,None,"“6000 GRADOS, VICENTE, TEMPERATURAS ALTAS PERO...",1,https://www.elmundotoday.com/2020/07/como-cada...
1,new_2,El Mundo Today,Tras veinte minutos intentando descifrar la le...,Xavi Puig,0,None,"""PUES NO SÉ, PÓNGASE UNA PELÍCULA DE ESTA CHIC...",1,https://www.elmundotoday.com/2020/07/tras-vein...
2,new_3,El Mundo Today,Acusan a una astróloga de hacer pronósticos be...,Javi Ramos,0,None,LOS CAPRICORNIO LLEVAN AÑOS VIVIENDO MUCHO MEJ...,1,https://www.elmundotoday.com/2020/07/acusan-a-...
3,new_4,El Mundo Today,Hacienda informó reiteradamente a la Audiencia...,Kike García,0,None,NO EXISTE NINGÚN TRABAJO PROBADO POR EL QUE PU...,1,https://www.elmundotoday.com/2020/07/hacienda-...
4,new_5,El Mundo Today,La Asociación Española de Berenjenas denuncia ...,Kike García,0,None,"“BASTA”, DICEN LAS BERENJENAS Tras creer duran...",1,https://www.elmundotoday.com/2020/07/la-asocia...
...,...,...,...,...,...,...,...,...,...
240,new_241,El Jueves,Miguel Bosé denuncia un complot para ocultar q...,None,None,None,'La expresión mirando a Cuenca se utiliza porq...,2,https://www.eljueves.es/news/miguel-bose-denun...
241,new_242,El Jueves,El rey Juan Carlos también pagó una despedida ...,None,None,None,"""No actué como rey. Solo como catador real"" ac...",2,https://www.eljueves.es/news/rey-juan-carlos-t...
242,new_243,El Jueves,El Apocalipsis maya se queda en casa después d...,None,None,None,"""Si es que ya es ensañarse"" declara Hace unos ...",2,https://www.eljueves.es/news/apocalipsis-maya-...
243,new_244,El Jueves,Llevar la mascarilla en el codo por la calle p...,None,None,None,Aunque inicialmente fue una práctica desacons...,2,https://www.eljueves.es/news/llevar-mascarilla...


In [10]:
len(df)

245

In [11]:
# Exportar a csv
nombre = 'fake_news_' + datetime.today().strftime('%d%m%Y') + '.csv'
df.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')

In [12]:
print('Cantidad de nulos en titulares: ', len(df[df['Título'].isnull()]))

Cantidad de nulos en titulares:  48
